In [3]:
# import packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy import signal

training = np.empty(shape=(0, 10))
testing = np.empty(shape=(0, 10))

df = pd.read_csv('dataset_1.txt', sep=',', header=None)
#print('deal with dataset ' + str(i + 1))
for c in range(1, 14):
    activity_data = df[df[24] == c].values
    # remove noise
    b, a = signal.butter(4, 0.04, 'low', analog=False)
    for j in range(24):
        activity_data[:, j] = signal.lfilter(b, a, activity_data[:, j])

    datat_len = len(activity_data)
    training_len = math.floor(datat_len * 0.8)
    training_data = activity_data[:training_len, :]
    testing_data = activity_data[training_len:, :]

    # data segementation: for time series data, we need to segment the whole time series, 
    # and then extract features from 
    # each period of time to represent the raw data. In this example code, we define each period of 
    # time contains 1000
    # data points. Each period of time contains different data points. You may consider overlap segmentation, which
    # means consecutive two segmentation share a part of data points, to get more feature samples.
    training_sample_number = training_len // 1000 + 1
    testing_sample_number = (datat_len - training_len) // 1000 + 1

    for s in range(training_sample_number):
        if s < training_sample_number - 1:
            sample_data = training_data[1000*s:1000*(s + 1), :]
        else:
            sample_data = training_data[1000*s:, :]

        # in this example code, only three accelerometer data in wrist sensor is used to extract three simple
        # features: min, max, and mean value in a period of time. Finally we get 9 features and 1 label
        # to construct feature dataset. You may consider all sensors' data and extract other features

        feature_sample = []
        for i in range(3):
            feature_sample.append(np.min(sample_data[:, i]))
            feature_sample.append(np.max(sample_data[:, i]))
            feature_sample.append(np.mean(sample_data[:, i]))
        feature_sample.append(sample_data[0, -1])
        feature_sample = np.array([feature_sample])
        training = np.concatenate((training, feature_sample), axis=0)

    for s in range(testing_sample_number):
        if s < training_sample_number - 1:
            sample_data = testing_data[1000*s:1000*(s + 1), :]
        else:
            sample_data = testing_data[1000*s:, :]

        feature_sample = []
        for i in range(3):
            feature_sample.append(np.min(sample_data[:, i]))
            feature_sample.append(np.max(sample_data[:, i]))
            feature_sample.append(np.mean(sample_data[:, i]))
        feature_sample.append(sample_data[0, -1])
        feature_sample = np.array([feature_sample])
        testing = np.concatenate((testing, feature_sample), axis=0)

df_training = pd.DataFrame(training)
df_testing = pd.DataFrame(testing)
df_training.to_csv('training_data.csv', index=None, header=None)
df_testing.to_csv('testing_data.csv', index=None, header=None)

